<a href="https://colab.research.google.com/github/gvikas79/Spark-Tutorials/blob/main/Pyspark_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, MapType
from datetime import date

# Assume spark is already created
spark = SparkSession.builder.appName("PracticeDataset").getOrCreate()

# Define the schema
schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("product", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", DoubleType(), True),
    StructField("order_date", DateType(), True),
    StructField("tags", StringType(), True), # For split() and regex examples
    StructField("details", StringType(), True), # For translate(), substring() examples
    StructField("features", StringType(), True), # For regexp_extract(), regexp_replace() examples
    StructField("product_attributes", MapType(StringType(), StringType()), True), # For MapType, explode, map_keys, map_values
    StructField("related_products", StringType(), True) # For collect_list, collect_set
])

# Sample data
data = [
    (1, 101, "Laptop", 1, 1200.00, date(2023, 1, 15), "electronics,office", "Serial: A1B2C3D4", "user_123_laptop", {"color": "silver", "brand": "XYZ"}, "Keyboard,Mouse,Monitor"),
    (2, 102, "Mouse", 2, 25.50, date(2023, 1, 15), "electronics,accessory", "Model: M500", "user_456_mouse", {"color": "black", "wireless": "true"}, "Laptop,Monitor"),
    (3, 101, "Keyboard", 1, 75.00, date(2023, 1, 16), "electronics,accessory", "SKU: KB-101", "id_789_keyboard", {"layout": "US", "mechanical": "false"}, "Mouse,Monitor"),
    (4, 103, "Monitor", 1, 300.00, date(2023, 1, 16), "electronics,display", "DisplaySize: 27inch", "user_123_monitor", {"size": "27", "resolution": "1080p"}, "Laptop,Keyboard"),
    (5, 102, "Desk Chair", 1, 150.00, date(2023, 1, 17), "furniture,office", "Weight: 20kg", "user_456_chair", {"material": "mesh", "adjustable": "true"}, "Desk,Lamp"),
    (6, 104, "Lamp", 2, 35.00, date(2023, 1, 17), "furniture,lighting", "BulbType: LED", "id_abc_lamp", None, "Desk Chair,Desk"), # Example with None map
    (7, 101, "Laptop", 1, 1200.00, date(2023, 1, 18), "electronics,office", "Serial: E5F6G7H8", "user_123_laptop", {"color": "silver", "brand": "XYZ"}, "Keyboard,Mouse,Monitor"), # Duplicate order
    (8, 105, "Notebook", 5, 3.00, date(2023, 1, 18), "office,stationery", "Pages: 100", "user_xyz_notebook", {}, "Pen,Pencil"), # Example with empty map
    (9, 103, "Desk", 1, 250.00, date(2023, 1, 19), "furniture,office", "Material: Wood", "user_789_desk", {"size": "medium"}, "Desk Chair,Lamp"),
    (10, 104, "Pen", 10, 1.50, date(2023, 1, 19), "office,stationery", "InkColor: Blue", "id_def_pen", {"color": "blue"}, "Notebook,Pencil"),
    (11, 105, "Pencil", 12, 0.50, date(2023, 1, 20), "office,stationery", "LeadSize: 0.7mm", "user_xyz_pencil", {"lead": "0.7"}, "Notebook,Pen"),
    (12, 106, "Tablet", 1, 400.00, date(2023, 1, 20), "electronics", "Model: Tab-Pro", "user_abc_tablet", {"os": "Android"}, None), # Example with None related_products
    (13, 106, "Protector", 1, 15.00, date(2023, 1, 20), "electronics,accessory", "Type: Screen", "user_abc_protector", {}, ""), # Example with empty related_products
    # Added new rows
    (14, 101, "Mouse", 1, 25.50, date(2023, 1, 21), "electronics,accessory", "Model: M600", "user_101_mouse", {"color": "white", "wireless": "false"}, "Keyboard"),
    (15, 102, "Laptop", 1, 1100.00, date(2023, 1, 21), "electronics,office", "Serial: I9J10K11L12", "user_102_laptop", {"color": "black", "brand": "UVW"}, "Mouse,Monitor"),
    (16, 103, "Keyboard", 2, 70.00, date(2023, 1, 22), "electronics,accessory", "SKU: KB-202", "user_103_keyboard", {"layout": "UK", "mechanical": "true"}, "Mouse"),
    (17, 104, "Desk", 1, 220.00, date(2023, 1, 22), "furniture,office", "Material: Metal", "user_104_desk", {"size": "large"}, "Chair"),
    (18, 105, "Lamp", 1, 30.00, date(2023, 1, 23), "furniture,lighting", "BulbType: Incandescent", "user_105_lamp", None, "Desk"),
    (19, 106, "Notebook", 3, 2.50, date(2023, 1, 23), "office,stationery", "Pages: 150", "user_106_notebook", {}, "Pen,Pencil"),
    (20, 101, "Monitor", 1, 280.00, date(2023, 1, 24), "electronics,display", "DisplaySize: 24inch", "user_101_monitor", {"size": "24", "resolution": "1080p"}, "Laptop")
]

practice_df = spark.createDataFrame(data, schema)

# Show the DataFrame and its schema
print("Sample Practice DataFrame:")
practice_df.show(truncate=False)
practice_df.printSchema()

Sample Practice DataFrame:
+--------+-----------+----------+--------+------+----------+---------------------+----------------------+------------------+--------------------------------------+----------------------+
|order_id|customer_id|product   |quantity|price |order_date|tags                 |details               |features          |product_attributes                    |related_products      |
+--------+-----------+----------+--------+------+----------+---------------------+----------------------+------------------+--------------------------------------+----------------------+
|1       |101        |Laptop    |1       |1200.0|2023-01-15|electronics,office   |Serial: A1B2C3D4      |user_123_laptop   |{color -> silver, brand -> XYZ}       |Keyboard,Mouse,Monitor|
|2       |102        |Mouse     |2       |25.5  |2023-01-15|electronics,accessory|Model: M500           |user_456_mouse    |{color -> black, wireless -> true}    |Laptop,Monitor        |
|3       |101        |Keyboard  |1    

In [2]:
from os import truncate
#Extract each individual tag from the tags column and create a new row for each tag, keeping the order_id.
from pyspark.sql.functions import col, explode, split

df_exploded_tags = practice_df.select(col("order_id"), explode(split(col("tags"), ",")).alias("tag"))
df_exploded_tags.show(truncate= False)

+--------+-----------+
|order_id|tag        |
+--------+-----------+
|1       |electronics|
|1       |office     |
|2       |electronics|
|2       |accessory  |
|3       |electronics|
|3       |accessory  |
|4       |electronics|
|4       |display    |
|5       |furniture  |
|5       |office     |
|6       |furniture  |
|6       |lighting   |
|7       |electronics|
|7       |office     |
|8       |office     |
|8       |stationery |
|9       |furniture  |
|9       |office     |
|10      |office     |
|10      |stationery |
+--------+-----------+
only showing top 20 rows



In [3]:
#Count how many orders belong to each unique tag.

df_tag_counts = df_exploded_tags.groupBy("tag").count()
df_tag_counts.show()


+-----------+-----+
|        tag|count|
+-----------+-----+
|     office|   10|
|    display|    2|
| stationery|    4|
|   lighting|    2|
|  furniture|    5|
|electronics|   11|
|  accessory|    5|
+-----------+-----+



In [4]:
#From the details column, extract the serial number (e.g., "A1B2C3D4") for products where the detail starts with "Serial: ".

from pyspark.sql.functions import regexp_extract

df_details_serial = practice_df.withColumn("ID", regexp_extract(col("details"), r"Serial: (.*)", 1))


In [5]:
df_details_serial.show()

+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+-----------+
|order_id|customer_id|   product|quantity| price|order_date|                tags|             details|          features|  product_attributes|    related_products|         ID|
+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+-----------+
|       1|        101|    Laptop|       1|1200.0|2023-01-15|  electronics,office|    Serial: A1B2C3D4|   user_123_laptop|{color -> silver,...|Keyboard,Mouse,Mo...|   A1B2C3D4|
|       2|        102|     Mouse|       2|  25.5|2023-01-15|electronics,acces...|         Model: M500|    user_456_mouse|{color -> black, ...|      Laptop,Monitor|           |
|       3|        101|  Keyboard|       1|  75.0|2023-01-16|electronics,acces...|         SKU: KB-101|   id_789_keyboard

In [6]:
#From the features column, extract the numeric ID (e.g., "123") that appears after "user_".

from pyspark.sql.functions import regexp_replace

df_replaced_user= practice_df.select(col("features"), regexp_replace(col("features"), "id_", "user_").alias("replaced_features"))
df_replaced_user.show()
df_user_extractID = df_replaced_user.withColumn("extracted_id", regexp_extract(col("replaced_features"), r"user_(.*)", 1))
df_user_extractID.show()

+------------------+------------------+
|          features| replaced_features|
+------------------+------------------+
|   user_123_laptop|   user_123_laptop|
|    user_456_mouse|    user_456_mouse|
|   id_789_keyboard| user_789_keyboard|
|  user_123_monitor|  user_123_monitor|
|    user_456_chair|    user_456_chair|
|       id_abc_lamp|     user_abc_lamp|
|   user_123_laptop|   user_123_laptop|
| user_xyz_notebook| user_xyz_notebook|
|     user_789_desk|     user_789_desk|
|        id_def_pen|      user_def_pen|
|   user_xyz_pencil|   user_xyz_pencil|
|   user_abc_tablet|   user_abc_tablet|
|user_abc_protector|user_abc_protector|
|    user_101_mouse|    user_101_mouse|
|   user_102_laptop|   user_102_laptop|
| user_103_keyboard| user_103_keyboard|
|     user_104_desk|     user_104_desk|
|     user_105_lamp|     user_105_lamp|
| user_106_notebook| user_106_notebook|
|  user_101_monitor|  user_101_monitor|
+------------------+------------------+

+------------------+------------------+

In [7]:
#Create a new column by removing all vowels (both uppercase and lowercase) from the details column.

from pyspark.sql.functions import translate
df_removed_vowels = practice_df.withColumn("removed_vowels", translate(col("details"), "aeiouAEIOU", ""))

df_removed_vowels.show()

+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+
|order_id|customer_id|   product|quantity| price|order_date|                tags|             details|          features|  product_attributes|    related_products|  removed_vowels|
+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+
|       1|        101|    Laptop|       1|1200.0|2023-01-15|  electronics,office|    Serial: A1B2C3D4|   user_123_laptop|{color -> silver,...|Keyboard,Mouse,Mo...|    Srl: 1B2C3D4|
|       2|        102|     Mouse|       2|  25.5|2023-01-15|electronics,acces...|         Model: M500|    user_456_mouse|{color -> black, ...|      Laptop,Monitor|       Mdl: M500|
|       3|        101|  Keyboard|       1|  75.0|2023-01-16|electronics,acces...|         SKU: 

In [8]:
#Extract the first 5 characters of the features column.
from pyspark.sql.functions import substring

df_first_5_chars = practice_df.withColumn("first_5_chars", substring(col("features"), 1, 5))
df_first_5_chars.show()


+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+-------------+
|order_id|customer_id|   product|quantity| price|order_date|                tags|             details|          features|  product_attributes|    related_products|first_5_chars|
+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+-------------+
|       1|        101|    Laptop|       1|1200.0|2023-01-15|  electronics,office|    Serial: A1B2C3D4|   user_123_laptop|{color -> silver,...|Keyboard,Mouse,Mo...|        user_|
|       2|        102|     Mouse|       2|  25.5|2023-01-15|electronics,acces...|         Model: M500|    user_456_mouse|{color -> black, ...|      Laptop,Monitor|        user_|
|       3|        101|  Keyboard|       1|  75.0|2023-01-16|electronics,acces...|         SKU: KB-101|   id_78

In [9]:
#Create a new column that combines the product and quantity columns into a single string like "Laptop (1)".

from pyspark.sql.functions import concat, lit

df_combined_product_quantity = practice_df.withColumn("product_quantity", concat(col("product"), lit(" ("), col("quantity"), lit(")")))
df_combined_product_quantity.show()


+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+
|order_id|customer_id|   product|quantity| price|order_date|                tags|             details|          features|  product_attributes|    related_products|product_quantity|
+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+
|       1|        101|    Laptop|       1|1200.0|2023-01-15|  electronics,office|    Serial: A1B2C3D4|   user_123_laptop|{color -> silver,...|Keyboard,Mouse,Mo...|      Laptop (1)|
|       2|        102|     Mouse|       2|  25.5|2023-01-15|electronics,acces...|         Model: M500|    user_456_mouse|{color -> black, ...|      Laptop,Monitor|       Mouse (2)|
|       3|        101|  Keyboard|       1|  75.0|2023-01-16|electronics,acces...|         SKU: 

In [10]:
#Combine the elements of the related_products string into a list using a comma as a separator (you might need split() first).

df_combined_product_quantity = practice_df.withColumn("related_products", concat( lit("["), col("related_products"), lit("]")))
df_combined_product_quantity.show()

+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+
|order_id|customer_id|   product|quantity| price|order_date|                tags|             details|          features|  product_attributes|    related_products|
+--------+-----------+----------+--------+------+----------+--------------------+--------------------+------------------+--------------------+--------------------+
|       1|        101|    Laptop|       1|1200.0|2023-01-15|  electronics,office|    Serial: A1B2C3D4|   user_123_laptop|{color -> silver,...|[Keyboard,Mouse,M...|
|       2|        102|     Mouse|       2|  25.5|2023-01-15|electronics,acces...|         Model: M500|    user_456_mouse|{color -> black, ...|    [Laptop,Monitor]|
|       3|        101|  Keyboard|       1|  75.0|2023-01-16|electronics,acces...|         SKU: KB-101|   id_789_keyboard|{layout -> US, me...|     [Mouse,Monitor]|
|       4|      

In [11]:
#Explode the product_attributes map to have one row per attribute key-value pair, keeping the order_id and product.

from pyspark.sql.functions import explode, map_keys, map_values, explode_outer

df_exploded_attributes = practice_df.select(col("order_id"), col("product"), explode_outer(col("product_attributes")).alias("att_key","att_values"))
df_exploded_attributes.show()


+--------+----------+----------+----------+
|order_id|   product|   att_key|att_values|
+--------+----------+----------+----------+
|       1|    Laptop|     color|    silver|
|       1|    Laptop|     brand|       XYZ|
|       2|     Mouse|     color|     black|
|       2|     Mouse|  wireless|      true|
|       3|  Keyboard|    layout|        US|
|       3|  Keyboard|mechanical|     false|
|       4|   Monitor|      size|        27|
|       4|   Monitor|resolution|     1080p|
|       5|Desk Chair|  material|      mesh|
|       5|Desk Chair|adjustable|      true|
|       6|      Lamp|      NULL|      NULL|
|       7|    Laptop|     color|    silver|
|       7|    Laptop|     brand|       XYZ|
|       8|  Notebook|      NULL|      NULL|
|       9|      Desk|      size|    medium|
|      10|       Pen|     color|      blue|
|      11|    Pencil|      lead|       0.7|
|      12|    Tablet|        os|   Android|
|      13| Protector|      NULL|      NULL|
|      14|     Mouse|     color|

In [13]:
#Get a list of all unique attribute keys present in the product_attributes column across all orders.

from pyspark.sql.functions import map_keys, map_values

df_unique_keys = practice_df.select(map_keys(col("product_attributes"))).distinct()
df_unique_keys.show()

+----------------------------+
|map_keys(product_attributes)|
+----------------------------+
|          [size, resolution]|
|        [material, adjust...|
|              [color, brand]|
|                        NULL|
|                          []|
|        [layout, mechanical]|
|           [color, wireless]|
|                     [color]|
|                      [size]|
|                        [os]|
|                      [lead]|
+----------------------------+



In [15]:
#Extract the value associated with the key "color" from the product_attributes map.

from pyspark.sql.functions import col

df_color_value = practice_df.select(col("order_id"), col("product"), col("product_attributes").getItem("color").alias("color_value"))
df_color_value.show()

+--------+----------+-----------+
|order_id|   product|color_value|
+--------+----------+-----------+
|       1|    Laptop|     silver|
|       2|     Mouse|      black|
|       3|  Keyboard|       NULL|
|       4|   Monitor|       NULL|
|       5|Desk Chair|       NULL|
|       6|      Lamp|       NULL|
|       7|    Laptop|     silver|
|       8|  Notebook|       NULL|
|       9|      Desk|       NULL|
|      10|       Pen|       blue|
|      11|    Pencil|       NULL|
|      12|    Tablet|       NULL|
|      13| Protector|       NULL|
|      14|     Mouse|      white|
|      15|    Laptop|      black|
|      16|  Keyboard|       NULL|
|      17|      Desk|       NULL|
|      18|      Lamp|       NULL|
|      19|  Notebook|       NULL|
|      20|   Monitor|       NULL|
+--------+----------+-----------+



In [16]:
#For each customer_id, create a list of all products they have ordered (collect_list).

from pyspark.sql.functions import collect_list

df_customer_products = practice_df.groupBy("customer_id").agg(collect_list("product").alias("products_ordered"))
df_customer_products.show()

+-----------+--------------------+
|customer_id|    products_ordered|
+-----------+--------------------+
|        101|[Laptop, Keyboard...|
|        103|[Monitor, Desk, K...|
|        102|[Mouse, Desk Chai...|
|        105|[Notebook, Pencil...|
|        104|   [Lamp, Pen, Desk]|
|        106|[Tablet, Protecto...|
+-----------+--------------------+



In [17]:
#For each customer_id, create a set of unique products they have ordered (collect_set).

from pyspark.sql.functions import collect_set

df_customer_unique_products = practice_df.groupBy("customer_id").agg(collect_set("product").alias("unique_products_ordered"))
df_customer_unique_products.show()

+-----------+-----------------------+
|customer_id|unique_products_ordered|
+-----------+-----------------------+
|        101|   [Keyboard, Laptop...|
|        103|   [Keyboard, Desk, ...|
|        102|   [Mouse, Laptop, D...|
|        105|   [Pencil, Notebook...|
|        104|      [Desk, Pen, Lamp]|
|        106|   [Notebook, Tablet...|
+-----------+-----------------------+



In [28]:
#Take a random sample of 20% of the rows from the DataFrame.

from pyspark.sql.functions import rand
from pyspark.sql.functions import rand

df_random_sample = practice_df.sample(withReplacement=False, fraction=0.2, seed=42)
df_random_sample.show()

+--------+-----------+--------+--------+-----+----------+-------------------+-------------------+-----------------+--------------------+----------------+
|order_id|customer_id| product|quantity|price|order_date|               tags|            details|         features|  product_attributes|related_products|
+--------+-----------+--------+--------+-----+----------+-------------------+-------------------+-----------------+--------------------+----------------+
|       8|        105|Notebook|       5|  3.0|2023-01-18|  office,stationery|         Pages: 100|user_xyz_notebook|                  {}|      Pen,Pencil|
|      20|        101| Monitor|       1|280.0|2023-01-24|electronics,display|DisplaySize: 24inch| user_101_monitor|{size -> 24, reso...|          Laptop|
+--------+-----------+--------+--------+-----+----------+-------------------+-------------------+-----------------+--------------------+----------------+



In [25]:
#Find the total quantity of products ordered for each unique tag.

from pyspark.sql.functions import col, explode, split, sum

df_tag_quantity = practice_df.select(col("order_id"), explode(split(col("tags"), ",")).alias("tag"), col("quantity"))
df_tag_quantity.show()
df_tag_quantity_sum = df_tag_quantity.groupBy("tag").agg(sum("quantity").alias("total_quantity"))
df_tag_quantity_sum.show()

+--------+-----------+--------+
|order_id|        tag|quantity|
+--------+-----------+--------+
|       1|electronics|       1|
|       1|     office|       1|
|       2|electronics|       2|
|       2|  accessory|       2|
|       3|electronics|       1|
|       3|  accessory|       1|
|       4|electronics|       1|
|       4|    display|       1|
|       5|  furniture|       1|
|       5|     office|       1|
|       6|  furniture|       2|
|       6|   lighting|       2|
|       7|electronics|       1|
|       7|     office|       1|
|       8|     office|       5|
|       8| stationery|       5|
|       9|  furniture|       1|
|       9|     office|       1|
|      10|     office|      10|
|      10| stationery|      10|
+--------+-----------+--------+
only showing top 20 rows

+-----------+--------------+
|        tag|total_quantity|
+-----------+--------------+
|     office|            36|
|    display|             2|
| stationery|            30|
|   lighting|             3|
|  